In [1]:
import pandas as pd
import numpy as np
import os

os.chdir("..")  # from /notebooks to project root


In [2]:
debts = pd.read_csv("data/cleaned/debts_clean.csv")
debts


,debt_id,account_id,current_balance,interest_rate,minimum_payment
0,D1,IUPM04409079772781,5200.0,22.9,160.0
1,D2,BLAT22216107051843,12400.0,7.2,275.0
2,D3,UTXA55295806601382,8600.0,9.9,210.0
3,D4,XICF70493862044851,3100.0,19.5,95.0
4,D5,KOSW19711121259020,17800.0,5.9,320.0


In [3]:
debts["current_balance"] = pd.to_numeric(debts["current_balance"], errors="coerce")
debts["interest_rate"] = pd.to_numeric(debts["interest_rate"], errors="coerce")
debts["minimum_payment"] = pd.to_numeric(debts["minimum_payment"], errors="coerce")

debts.isna().sum()


debt_id            0
account_id         0
current_balance    0
interest_rate      0
minimum_payment    0
dtype: int64

In [4]:
monthly_budget = 800.0   # baseline assumption; can be parameterized later


In [5]:
from copy import deepcopy

def simulate_payoff(debts_df: pd.DataFrame, monthly_budget: float, strategy: str, max_months: int = 600) -> pd.DataFrame:
    """
    Simulate month-by-month debt payoff.

    debts_df must contain:
      - debt_id
      - current_balance
      - interest_rate (APR percent)
      - minimum_payment
    strategy: "snowball" (smallest balance first) or "avalanche" (highest APR first)
    """
    df = debts_df.copy()
    df["balance"] = df["current_balance"].astype(float)
    df["apr"] = df["interest_rate"].astype(float)
    df["min_pay"] = df["minimum_payment"].astype(float)

    # For tracking
    month_rows = []
    total_interest_paid = 0.0

    for month in range(1, max_months + 1):
        # Stop if all balances are paid
        if (df["balance"] <= 0.01).all():
            break

        # Apply monthly interest to each remaining balance
        df["monthly_rate"] = (df["apr"] / 100.0) / 12.0
        interest = df["balance"].clip(lower=0) * df["monthly_rate"]
        df["balance"] = df["balance"] + interest
        total_interest_paid += interest.sum()

        # Determine minimum payments for active debts
        active = df["balance"] > 0.01
        min_due = df.loc[active, "min_pay"].sum()

        # If budget is less than minimums, pay proportionally (edge case)
        budget = float(monthly_budget)
        payments = pd.Series(0.0, index=df.index)

        if budget <= min_due + 1e-9:
            # Pro-rate payments across active debts
            weights = df.loc[active, "min_pay"] / min_due
            payments.loc[active] = budget * weights
        else:
            # Pay minimums first
            payments.loc[active] = df.loc[active, "min_pay"]
            extra = budget - min_due

            # Pick target debt for extra payment based on strategy
            candidates = df.loc[active].copy()

            if strategy.lower() == "snowball":
                # smallest balance first
                target_idx = candidates["balance"].idxmin()
            elif strategy.lower() == "avalanche":
                # highest APR first (tie-breaker: highest balance)
                target_idx = candidates.sort_values(["apr", "balance"], ascending=[False, False]).index[0]
            else:
                raise ValueError("strategy must be 'snowball' or 'avalanche'")

            payments.loc[target_idx] += extra

        # Apply payments (cannot pay more than balance)
        actual_payments = payments.clip(lower=0)
        actual_payments = actual_payments.where(actual_payments <= df["balance"], df["balance"])

        df["balance"] = df["balance"] - actual_payments

        # Record month snapshot
        row = {
            "month": month,
            "strategy": strategy.lower(),
            "total_balance": float(df["balance"].clip(lower=0).sum()),
            "total_interest_paid_to_date": float(total_interest_paid),
            "total_payment": float(actual_payments.sum()),
        }
        month_rows.append(row)

    return pd.DataFrame(month_rows)


In [6]:
sim_snowball = simulate_payoff(debts, monthly_budget=monthly_budget, strategy="snowball")
sim_snowball.head(), sim_snowball.tail()


(   month  strategy  total_balance  total_interest_paid_to_date  total_payment
 0      1  snowball   46682.475000                   382.475000          800.0
 1      2  snowball   46261.914576                   761.914576          800.0
 2      3  snowball   45838.290785                  1138.290785          800.0
 3      4  snowball   45411.575349                  1511.575349          800.0
 4      5  snowball   44981.739645                  1881.739645          800.0,
     month  strategy  total_balance  total_interest_paid_to_date  total_payment
 78     79  snowball    2520.337365                 17995.541828     553.091925
 79     80  snowball    1732.729024                 18007.933487     800.000000
 80     81  snowball     941.248275                 18016.452738     800.000000
 81     82  snowball     145.876079                 18021.080542     800.000000
 82     83  snowball       0.000000                 18021.797766     146.593303)

In [7]:
sim_snowball.iloc[-1]


month                                    83
strategy                           snowball
total_balance                           0.0
total_interest_paid_to_date    18021.797766
total_payment                    146.593303
Name: 82, dtype: object

In [8]:
sim_avalanche = simulate_payoff(debts, monthly_budget=monthly_budget, strategy="avalanche")

summary = pd.DataFrame([
    {
        "strategy": "snowball",
        "months_to_payoff": int(sim_snowball["month"].max()),
        "total_interest_paid": float(sim_snowball["total_interest_paid_to_date"].iloc[-1]),
        "monthly_budget": monthly_budget
    },
    {
        "strategy": "avalanche",
        "months_to_payoff": int(sim_avalanche["month"].max()),
        "total_interest_paid": float(sim_avalanche["total_interest_paid_to_date"].iloc[-1]),
        "monthly_budget": monthly_budget
    }
]).sort_values("total_interest_paid")

summary


,strategy,months_to_payoff,total_interest_paid,monthly_budget
1,avalanche,82,17996.472169,800.0
0,snowball,83,18021.797766,800.0


In [9]:
def simulate_payoff_detailed(debts_df: pd.DataFrame, monthly_budget: float, strategy: str, max_months: int = 600):
    df = debts_df.copy()
    df["balance"] = df["current_balance"].astype(float)
    df["apr"] = df["interest_rate"].astype(float)
    df["min_pay"] = df["minimum_payment"].astype(float)
    df["monthly_rate"] = (df["apr"] / 100.0) / 12.0

    # Track payoff month per debt
    payoff_month = {debt_id: None for debt_id in df["debt_id"].astype(str).tolist()}

    month_rows = []
    total_interest_paid = 0.0

    for month in range(1, max_months + 1):
        if (df["balance"] <= 0.01).all():
            break

        # Interest accrual
        interest = df["balance"].clip(lower=0) * df["monthly_rate"]
        df["balance"] = df["balance"] + interest
        total_interest_paid += float(interest.sum())

        active = df["balance"] > 0.01
        min_due = float(df.loc[active, "min_pay"].sum())
        budget = float(monthly_budget)

        payments = pd.Series(0.0, index=df.index)

        if budget <= min_due + 1e-9:
            weights = df.loc[active, "min_pay"] / min_due
            payments.loc[active] = budget * weights
        else:
            payments.loc[active] = df.loc[active, "min_pay"]
            extra = budget - min_due

            candidates = df.loc[active].copy()

            if strategy.lower() == "snowball":
                target_idx = candidates["balance"].idxmin()
            elif strategy.lower() == "avalanche":
                target_idx = candidates.sort_values(["apr", "balance"], ascending=[False, False]).index[0]
            else:
                raise ValueError("strategy must be 'snowball' or 'avalanche'")

            payments.loc[target_idx] += extra

        # Apply payments capped by remaining balance
        actual_payments = payments.clip(lower=0)
        actual_payments = actual_payments.where(actual_payments <= df["balance"], df["balance"])
        df["balance"] = df["balance"] - actual_payments

        # Record payoff month for debts that just reached ~0
        for _, r in df.iterrows():
            did = str(r["debt_id"])
            if payoff_month[did] is None and r["balance"] <= 0.01:
                payoff_month[did] = month

        # Store a monthly snapshot including balances per debt
        row = {
            "month": month,
            "strategy": strategy.lower(),
            "total_balance": float(df["balance"].clip(lower=0).sum()),
            "total_interest_paid_to_date": float(total_interest_paid),
            "total_payment": float(actual_payments.sum())
        }

        # Add each debt balance as a column (wide format)
        for _, r in df.iterrows():
            row[f"balance_{str(r['debt_id'])}"] = float(max(r["balance"], 0.0))

        month_rows.append(row)

    schedule_df = pd.DataFrame(month_rows)

    payoff_df = pd.DataFrame([
        {"strategy": strategy.lower(), "debt_id": k, "payoff_month": v}
        for k, v in payoff_month.items()
    ]).sort_values(["payoff_month", "debt_id"])

    return schedule_df, payoff_df


In [10]:
schedule_snowball, payoff_snowball = simulate_payoff_detailed(debts, monthly_budget, "snowball")
schedule_avalanche, payoff_avalanche = simulate_payoff_detailed(debts, monthly_budget, "avalanche")

payoff_snowball, payoff_avalanche


(   strategy debt_id  payoff_month
 2  snowball      D3            73
 1  snowball      D2            74
 3  snowball      D4            75
 0  snowball      D1            79
 4  snowball      D5            83,
     strategy debt_id  payoff_month
 2  avalanche      D3            73
 1  avalanche      D2            74
 3  avalanche      D4            75
 0  avalanche      D1            78
 4  avalanche      D5            82)

In [11]:
final_summary = pd.DataFrame([
    {
        "strategy": "snowball",
        "months_to_payoff": int(schedule_snowball["month"].max()),
        "total_interest_paid": float(schedule_snowball["total_interest_paid_to_date"].iloc[-1]),
        "monthly_budget": monthly_budget
    },
    {
        "strategy": "avalanche",
        "months_to_payoff": int(schedule_avalanche["month"].max()),
        "total_interest_paid": float(schedule_avalanche["total_interest_paid_to_date"].iloc[-1]),
        "monthly_budget": monthly_budget
    }
]).sort_values(["total_interest_paid", "months_to_payoff"])

final_summary


,strategy,months_to_payoff,total_interest_paid,monthly_budget
1,avalanche,82,17996.472169,800.0
0,snowball,83,18021.797766,800.0


In [12]:
import os
os.makedirs("outputs", exist_ok=True)


In [13]:
final_summary.to_csv("outputs/subtask_d_strategy_comparison.csv", index=False)
payoff_snowball.to_csv("outputs/subtask_d_payoff_months_snowball.csv", index=False)
payoff_avalanche.to_csv("outputs/subtask_d_payoff_months_avalanche.csv", index=False)
schedule_snowball.to_csv("outputs/subtask_d_monthly_schedule_snowball.csv", index=False)
schedule_avalanche.to_csv("outputs/subtask_d_monthly_schedule_avalanche.csv", index=False)

[os.path.exists(p) for p in [
    "outputs/subtask_d_strategy_comparison.csv",
    "outputs/subtask_d_payoff_months_snowball.csv",
    "outputs/subtask_d_payoff_months_avalanche.csv",
    "outputs/subtask_d_monthly_schedule_snowball.csv",
    "outputs/subtask_d_monthly_schedule_avalanche.csv",
]]


[True, True, True, True, True]